In [ ]:
from pyspark.sql import SparkSession
import random
import pandas as pd
from pyspark.sql import functions as F
spark = SparkSession.builder \.appName("ResultManagementSystem") \.getOrCreate()
def generate_student_profiles(num_students=10000):
    names = [f'Student{i}' for i in range(num_students)]
    return names
subjects = ["Electronics", "Programming", "Database", "Data Science", "Mathematics", "DSA"]

def generate_marks(num_students=10000, subjects=subjects):
    marks = {subject: [random.randint(0, 100) for _ in range(num_students)] for subject in subjects}
    return pd.DataFrame(marks)

student_names = generate_student_profiles()
marks_df = generate_marks()
marks_df['Student Name'] = student_names
spark_df = spark.createDataFrame(marks_df)

average_marks = spark_df.select([spark_df[subject].alias(subject) for subject in subjects]) \.agg(*[F.avg(subject).alias(f"avg_{subject}") for subject in subjects])
min_max_expressions = [F.min(subject).alias(f"min_{subject}") for subject in subjects] + \[F.max(subject).alias(f"max_{subject}") for subject in subjects]
min_max_marks = spark_df.select([spark_df[subject].alias(subject) for subject in subjects]) \.agg(*min_max_expressions)

spark_df = spark_df.withColumn("Total Marks", sum(spark_df[subject] for subject in subjects))
spark_df = spark_df.withColumn("Percentage", (F.col("Total Marks") / (len(subjects) * 100)) * 100)
spark_df.show()
average_marks.show()
min_max_marks.show()
spark.stop()


+-----------+-----------+--------+------------+-----------+---+------------+-----------+------------------+
|Electronics|Programming|Database|Data Science|Mathematics|DSA|Student Name|Total Marks|        Percentage|
+-----------+-----------+--------+------------+-----------+---+------------+-----------+------------------+
|          5|         64|      26|          36|         29| 21|    Student0|        181|30.166666666666668|
|          3|         62|      96|          24|         67| 51|    Student1|        303|              50.5|
|          9|         50|      37|          38|         32|  9|    Student2|        175|29.166666666666668|
|         87|         54|      63|          31|         82| 87|    Student3|        404| 67.33333333333333|
|         83|         79|      47|           0|         73| 23|    Student4|        305| 50.83333333333333|
|         48|         50|      86|          74|         15| 10|    Student5|        283| 47.16666666666667|
|         32|         82|   